In [ ]:
class SVR_general_cvxpy_m:
    
    def __init__(self, C = 0.1, epsilon = 0.01, mu = 0.5, lmbda = 0.5, kernel = "linear", **kernel_param):
        import cvxpy as cp
        import numpy as np
        from sklearn.metrics.pairwise import pairwise_kernels
        self.cp = cp
        self.C = C
        self.mu = mu
        self.lmbda = lmbda
        self.epsilon = epsilon
        self.kernel = kernel
        self.pairwise_kernels = pairwise_kernels
        self.kernel_param = kernel_param
        
    def fit(self, X, y):
        # hyperparameters
        C = self.C 
        epsilon =  self.epsilon
        mu = self.mu
        lmbda = self.lmbda
        kernel = self.kernel
        pairwise_kernels = self.pairwise_kernels
        cp = self.cp
        # Useful parameters
        ydim = y.shape[0]
        onev = np.ones((ydim,1))
        
        # Matrices for the optimizer
        K = mu*pairwise_kernels(X, X, metric = "linear") \
        + lmbda*pairwise_kernels(X, X, metric = kernel, **self.kernel_param)
        A = onev.T
        b = 0
        G = np.concatenate((np.identity(ydim), -np.identity(ydim)))
        h_ = np.concatenate((100*C*np.ones(ydim)/y, 100*C*np.ones(ydim)/y)); 
        h = h_.reshape(-1, 1)
        
        # loss function and constraints
        beta = cp.Variable((ydim,1))
        Ev = (y*epsilon)/100 
        objective = cp.Minimize((1/2)*cp.quad_form(beta, K) + Ev.T @ cp.abs(beta) - y.T @ beta)
        constraints = [A @ beta == b, G @ beta <= h]
        
        # Solver and solution
        prob = cp.Problem(objective,constraints)
        result = prob.solve()
        
        # support vectors
        beta_1 = np.array(beta.value).reshape(-1)
        beta_n = np.abs(beta_1)/beta_1.max()
        indx = beta_n > 0.01
        beta_sv = beta_1[indx]
        x_sv = X[indx,:]
        y_sv = y[indx]
        
        # get w_phi and b
        k_sv = mu*pairwise_kernels(x_sv, x_sv, metric = "linear") \
        + lmbda*pairwise_kernels(x_sv, x_sv, metric = kernel, **self.kernel_param)
        
        cons = np.where(beta_sv >= 0, 1 - epsilon/100, 1 + epsilon/100)
        
        w_phi = beta_sv @ k_sv
        b = np.mean((y_sv*cons - w_phi)); self.b = b
        self.beta_sv = beta_sv; self.x_sv = x_sv
        return self
        
    def predict(self, X_):
        k_test = self.mu*self.pairwise_kernels(x_sv, X_, metric = "linear") \
        + self.lmbda*self.pairwise_kernels(x_sv, x_sv, metric = self.kernel, **self.kernel_param)
        
        w_phi_test = self.beta_sv @ k_test
        predict = w_phi_test + self.b
        return predict
    
    def coef_(self):
        return self.beta_sv

In [ ]:
class SVR_general_cvxpy_ml:
    
    def __init__(self, C = 0.1, epsilon = 0.01, mu = 0.5, lmbda = 0.5, kernel = "linear", **kernel_param):
        import cvxpy as cp
        import numpy as np
        from sklearn.metrics.pairwise import pairwise_kernels
        self.cp = cp
        self.C = C
        self.mu = mu
        self.lmbda = lmbda
        self.epsilon = epsilon
        self.kernel = kernel
        self.pairwise_kernels = pairwise_kernels
        self.kernel_param = kernel_param
        
    def fit(self, X, y):
        # hyperparameters
        C = self.C 
        epsilon =  self.epsilon
        mu = self.mu
        lmbda = self.lmbda
        kernel = self.kernel
        pairwise_kernels = self.pairwise_kernels
        cp = self.cp
        # Useful parameters
        ydim = y.shape[0]
        onev = np.ones((ydim,1))
        
        # Matrices for the optimizer
        K = mu*pairwise_kernels(X, X, metric = "linear") \
        + lmbda*pairwise_kernels(X, X, metric = kernel, **self.kernel_param)
        A = onev.T
        b = 0
        G = np.concatenate((np.identity(ydim), -np.identity(ydim)))
        h_ = np.concatenate((100*C*np.ones(ydim)/y, 100*C*np.ones(ydim)/y)); 
        h = h_.reshape(-1, 1)
        
        # loss function and constraints
        beta = cp.Variable((ydim,1))
        Ev = (y*epsilon)/100 
        objective = cp.Minimize((1/2)*cp.quad_form(beta, K) + Ev.T @ cp.abs(beta) - y.T @ beta)
        constraints = [A @ beta == b, G @ beta <= h]
        
        # Solver and solution
        prob = cp.Problem(objective,constraints)
        result = prob.solve()
        
        # support vectors
        beta_1 = np.array(beta.value).reshape(-1)
        beta_n = np.abs(beta_1)/beta_1.max()
        indx = beta_n > 0.01
        beta_sv = beta_1[indx]
        x_sv = X[indx,:]
        y_sv = y[indx]
        
        # get w_phi and b
        k_sv = mu*pairwise_kernels(x_sv, x_sv, metric = "linear") \
        + lmbda*pairwise_kernels(x_sv, x_sv, metric = kernel, **self.kernel_param)
        
        cons = np.where(beta_sv >= 0, 1 - epsilon/100, 1 + epsilon/100)
        
        w_phi = beta_sv @ k_sv
        b = np.mean((y_sv*cons - w_phi)); self.b = b
        self.beta_sv = beta_sv; self.x_sv = x_sv
        return self
        
    def predict(self, X_):
        k_test = self.mu*self.pairwise_kernels(x_sv, X_, metric = "linear") \
        + self.lmbda*self.pairwise_kernels(x_sv, x_sv, metric = self.kernel, **self.kernel_param)
        
        w_phi_test = self.beta_sv @ k_test
        predict = w_phi_test + self.b
        return predict
    
    def coef_(self):
        return self.beta_sv